In [1]:
import pandas as pd
import re
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [13]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats_v2.csv")
player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84,0,1.0,0.0,0,1,1.0,100.0,1.0,1,55.6,5,0.0,0.0,0.0,4.0,88.9,9,31,8.0,5.0,0.0,0,1,0.0,0,0,3.0,3,0,0.0,0.0,0.0,0,1.0,4,83.3,0.0,0.0,6,2.0,2,0,1.0,172.0,1,5.0,6.1,2,0.0,4,1.0,0,1.0,1,0,0,1,9,2,4,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,NaN,0.000000,-0.235667,0.000000,1.080668,0.318364,0.762304,6.534905,3.036248,3.498657,-0.461340,0.190552,0.545439,1.933806
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87,0,0.0,0.0,0,4,1.0,100.0,1.0,0,66.7,8,0.0,0.0,0.0,5.0,83.3,12,36,10.0,5.0,0.0,2,0,0.0,1,0,8.0,6,0,0.0,0.0,0.0,2,0.0,7,83.3,1.0,1.0,6,1.0,0,0,0.0,258.0,2,7.0,8.0,1,0.0,2,1.0,0,2.0,0,3,1,0,12,6,4,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,NaN,0.000000,-0.075963,0.010892,0.458517,0.623663,-0.165146,2.267625,3.311832,-1.044207,-0.487945,-0.039690,0.266572,1.916089
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77,0,0.0,9.0,3,0,0.0,0.0,0.0,0,50.0,5,0.0,0.0,0.0,3.0,70.0,10,39,7.0,4.0,0.0,0,2,0.0,1,0,1.0,3,0,0.0,0.0,0.0,3,0.0,1,57.1,0.0,0.0,7,2.3,2,1,2.0,207.0,1,9.0,10.1,0,0.0,4,1.0,0,0.0,0,1,0,3,10,0,5,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,NaN,0.000000,0.460322,-0.166355,0.734292,0.734427,-0.000135,6.061680,6.251205,-0.189524,0.197894,-0.035491,1.216250,5.936939
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncraig (WA)/Subiaco (WAFL),7.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1990-06-07,84,0,0.0,0.0,0,4,1.0,50.0,2.0,0,12.5,2,0.0,0.0,0.0,5.0,60.9,23,95,14.0,12.0,0.0,0,0,0.0,0,0,2.0,2,0,0.0,0.0,0.0,2,3.0,6,57.1,7.0,7.0,21,10.5,8,0,0.0,641.0,4,8.0,6.3,11,0.0,1,0.0,0,4.0,0,1,0,0,16,6,14,NaN,NaN,B

In [14]:
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

In [15]:
match = player_stats[player_stats['Match_ID'] == "202323_WesternBulldogs_WestCoast"]
match.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent
28081,202323_WesternBulldogs_WestCoast,West Coast,Alex Witherden,202323,Alex_Witherden,MEDIUM_DEFENDER,CD_I998128,24,188,85,23,RIGHT,VIC,2016.0,2017.0,"St Joseph&apos;s, Geel (Vic)/Geel College (Vic...",23.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-09-10,93,0,1.0,0.0,0,3,0.0,0.0,0.0,0,44.4,8,0.0,0.0,0.0,7.0,90.9,22,73,20.0,11.0,0.0,0,3,0.0,0,0,3.0,9,0,0.0,0.0,0.0,0,0.0,8,84.6,6.0,6.0,13,1.4,3,0,0.0,377.0,5,7.0,11.8,8,0.0,3,2.0,0,3.0,0,1,0,0,18,4,10,NaN,NaN,Back Pocket Right,FINAL_TEAM,Back-Pocket,Back,NaN,NaN,2023,0.000000,-0.267964,0.023892,0.487451,0.280099,0.207352,1.037687,1.599591,-0.561904,-1.010768,0.023430,-0.087556,-0.381674,Alex Witherden_2023,0,0.000000,Western Bulldogs,West Coast,Western Bulldogs
28082,202323_WesternBulldogs_WestCoast,West Coast,Andrew Gaff,202323,Andrew_Gaff,MIDFIELDER,CD_I290801,31,184,83,3,LEFT,VIC,2010.0,2011.0,Kew (Vic)/Carey Grammar (Vic)/Oakleigh U18,4.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1992-06-16,86,0,1.0,6.0,0,2,0.0,0.0,0.0,0,22.7,5,0.0,0.0,0.0,16.0,78.3,23,99,18.0,8.0,1.0,0,0,0.0,0,0,5.0,12,0,0.0,0.0,0.0,3,0.0,3,72.7,0.0,0.0,11,0.9,6,0,0.0,218.0,0,21.0,5.1,3,0.0,4,0.0,1,0.0,1,6,0,1,22,3,17,NaN,NaN,Wing Left,FINAL_TEAM,Wing,Midfield,NaN,NaN,2023,2.148563,0.191014,0.052414,-0.063434,0.129387,-0.192821,0.607441,1.700765,-1.093324,-0.230022,0.138102,0.589806,3.155958,Andrew Gaff_2023,0,-2.148563,Western Bulldogs,West Coast,Western Bulldogs
28083,202323_WesternBulldogs_WestCoast,West Coast,Bailey J. Williams,202323,Bailey_J._Williams,RUCK,CD_I1006114,23,201,101,32,RIGHT,VIC,2018.0,2020.0,Pines (Vic)/Western Port SC (Vic)/Dandenong U18,35.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2000-04-17,81,0,0.0,26.0,3,4,0.0,0.0,0.0,0,90.9,10,0.0,0.0,0.0,6.0,50.0,10,74,5.0,3.0,1.0,2,2,0.0,0,0,6.0,4,41,9.0,22.0,41.8,2,0.0,1,50.0,0.0,0.0,6,1.5,1,0,0.0,126.0,2,14.0,8.7,0,98.0,3,3.0,0,1.0,2,2,0,5,11,4,1,NaN,NaN,Ruck,FINAL_TEAM,Ruck,Ruck,NaN,NaN,2023,0.000000,0.251066,0.067486,0.613062,0.577126,0.035936,4.382892,4.323413,0.059479,-0.014674,0.022624,0.359049,3.785106,Bailey J. Williams_2023,0,0.000000,Western Bulldogs,West Coast,Western Bulldogs
28084,202323_WesternBulldogs_WestCoast,West Coast,Campbell Chesser,202323,Campbell_Chesser,MIDFIELDER,CD_I1017057,20,186,85,18,RIGHT,NSW,2021.0,2023.0,Lavington (NSW)/Melbourne Grammar (Vic)/Sandri...,14.0,nationalDraft,

In [16]:
match_for = match.groupby('Team').sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
match_for.columns = [x+"_F" for x in list(match_for)]
match_against = match.groupby(['Opponent']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
match_against.columns = [x+"_A" for x in list(match_against)]
match_summary = pd.concat([match_for, match_against], axis=1)
for col in [x.replace("_F", "") for x in list(match_for)]:
        match_summary[col+"_diff"] = match_summary[col+"_F"] - match_summary[col+"_A"]
match_summary

,Score_F,xScore_F,vaep_value_F,vaep_value_received_F,offensive_value_F,defensive_value_F,exp_vaep_value_F,exp_vaep_value_received_F,exp_offensive_value_F,exp_defensive_value_F,Score_A,xScore_A,vaep_value_A,vaep_value_received_A,offensive_value_A,defensive_value_A,exp_vaep_value_A,exp_vaep_value_received_A,exp_offensive_value_A,exp_defensive_value_A,Score_diff,xScore_diff,vaep_value_diff,vaep_value_received_diff,offensive_value_diff,defensive_value_diff,exp_vaep_value_diff,exp_vaep_value_received_diff,exp_offensive_value_diff,exp_defensive_value_diff
West Coast,92,83.056271,15.951490,14.620448,15.680665,0.270826,80.338424,91.128484,85.051029,-4.712605,83,82.120013,13.929938,15.860156,12.661023,1.268915,80.989393,87.702119,80.727490,0.261904,9,0.936258,2.021552,-1.239708,3.019642,-0.99809,-0.650969,3.426365,4.32354,-4.97451
Western Bulldogs,83,82.120013,13.929938,15.860156,12.661023,1.268915,80.989393,87.702119,80.727490,0.261904,92,83.056271,15.951490,14.620448,15.680665,0.270826,80.338424,91.128484,85.051029,-4.712605,-9,-0.936258,-2.021552,1.239708,-3.019642,0.99809,0.650969,-3.426365,-4.32354,4.97451


In [17]:
home_team = match[match['Team'] == match['Home_Team']]
away_team = match[match['Team'] != match['Home_Team']]

home_team['exp_vaep_value_%'] = home_team['exp_vaep_value'] / home_team['exp_vaep_value'].sum()
home_team['player_value'] = home_team['exp_vaep_value_%'] * (home_team['xScore'].sum())

away_team['exp_vaep_value_%'] = away_team['exp_vaep_value'] / away_team['exp_vaep_value'].sum()
away_team['player_value'] = away_team['exp_vaep_value_%'] * (away_team['xScore'].sum())
match_player_values = pd.concat([home_team, away_team])
match_player_values[['Player', 'Team', 'Position_Group', 'Score', 'xScore', 'exp_vaep_value', 'exp_vaep_value_%', 'player_value']].sort_values(by='player_value', ascending=False).head(20)

,Player,Team,Position_Group,Score,xScore,exp_vaep_value,exp_vaep_value_%,player_value
28121,Marcus Bontempelli,Western Bulldogs,Midfield,6,3.469234,19.647461,0.242593,19.921741
28086,Elliot Yeo,West Coast,NaN,6,3.583849,11.569283,0.144007,11.960671
28101,Tim Kelly,West Coast,Midfield,1,3.229225,10.099748,0.125715,10.441423
28111,Caleb Daniel,Western Bulldogs,Interchange,0,0.000000,10.257896,0.126657,10.401097
28093,Jeremy McGovern,West Coast,Back,0,0.000000,7.999404,0.099571,8.270024
28123,Rhylee West,Western Bulldogs,Forward,7,9.770643,7.634124,0.094261,7.740697
28114,Ed Richards,Western Bulldogs,Back,0,0.000000,5.671801,0.070031,5.750980
28091,Jamie Cripps,West Coast,Forward,30,14.786997,5.419935,0.067464,5.603291
28110,Bailey Williams,Western Bulldogs,Midfield,1,2.164810,5.313209,0.065604,5.387382
28096,Luke Edwards,West Coast,NaN,0,3.777294,5.127543,0.063824,5.301008


In [18]:
def create_player_value(match_player_stats):
    
    home_team = match_player_stats[match_player_stats['Team'] == match_player_stats['Home_Team']]
    home_team['exp_vaep_value_%'] = home_team['exp_vaep_value'] / home_team['exp_vaep_value'].sum()
    home_team['player_value'] = home_team['exp_vaep_value_%'] * home_team['xScore'].sum()

    away_team = match_player_stats[match_player_stats['Team'] != match_player_stats['Home_Team']]
    away_team['exp_vaep_value_%'] = away_team['exp_vaep_value'] / away_team['exp_vaep_value'].sum()
    away_team['player_value'] = away_team['exp_vaep_value_%'] * away_team['xScore'].sum()
    match_player_values = pd.concat([home_team, away_team])
    match_player_values[['Player', 'Team', 'Position_Group', 'Score', 'xScore', 'exp_vaep_value', 'player_value']].sort_values(by='player_value', ascending=False).head(10)

    return match_player_values

In [19]:
match_player_values_list = []
for match_id in list(player_stats['Match_ID'].unique()):
    
    match_player_stats = player_stats[player_stats['Match_ID'] == match_id]
    match_player_values = create_player_value(match_player_stats)
    match_player_values_list.append(match_player_values)
    
player_values = pd.concat(match_player_values_list)


In [20]:
player_values_2023 = player_values[player_values['Season'] == 2023]
player_values_2023.sample(5)

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,exp_vaep_value_%,player_value
21406,202306_Hawthorn_Adelaide,Hawthorn,Tyler Brockman,202306,Tyler_Brockman,MEDIUM_FORWARD,CD_I1013511,20,180,80,33,RIGHT,WA,2020.0,2021.0,Wembley Downs (WA)/Subiaco (WAFL),46.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2002-11-22,71,0,1.0,0.0,0,1,0.0,0.0,0.0,0,20.0,2,0.0,0.0,0.0,2.0,80.0,10,46,8.0,3.0,1.0,0,0,100.0,1,2,2.0,6,0,0.0,0.0,0.0,1,0.0,1,75.0,0.0,0.0,4,0.7,2,1,1.0,177.0,1,4.0,9.5,0,0.0,4,0.0,2,1.0,0,1,0,0,10,2,8,NaN,NaN,Full Forward,FINAL_TEAM,Key-Forward,Forward,NaN,NaN,2023,8.422852,0.495040,0.000791,1.649444,1.736105,-0.086662,7.288368,8.153228,-0.864860,0.107685,0.007623,1.138537,5.291015,Tyler Brockman_2023,12,3.577148,Hawthorn,Adelaide,Adelaide,0.088571,7.405608
26116,202319_Carlton_WestCoast,Carlton,Matthew Cottrell,202319,Matthew_Cottrell,MIDFIELDER_FORWARD,CD_I1008154,23,184,83,46,RIGHT,VIC,2021.0,2020.0,Pearcedale (Vic)/Dandenong U18,23.0,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2000-02-29,96,2,0.0,NaN,0,1,NaN,NaN,NaN,0,NaN,4,NaN,NaN,NaN,NaN,84.6,13,66,NaN,NaN,NaN,0,1,0.0,2,0,NaN,8,0,NaN,NaN,NaN,1,NaN,2,NaN,NaN,NaN,5,NaN,4,1,NaN,157.0,1,NaN,NaN,0,NaN,6,NaN,2,NaN,0,5,2,0,13,2,9,NaN,NaN,Half Forward Flank Right,FINAL_TEAM,Half-Forward,Forward,NaN,NaN,2023,7.024005,-0.471889,0.000000,0.084892,0.028676,0.056216,6.110805,5.410023,0.700782,-0.057691,-0.022678,1.221756,4.547529,Matthew Cottrell_2023,2,-5.024005,Carlton,West Coast,West Coast,0.048607,6.355254
23085,202310_PortAdelaide_Melbourne,Melbourne,Trent Rivers,202310,Trent_Rivers,MEDIUM_DEFENDER,CD_I1009378,21,188,87,24,RIGHT,WA,2019.0,2020.0,Willetton (WA)/East Fremantle (WAFL),32.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2001-07-30,81,0,1.0,0.0,0,4,1.0,25.0,4.0,0,31.3,5,0.0,0.0,0.0,9.0,57.9,19,77,11.0,10.0,0.0,1,1,0.0,0,0,4.0,3,0,0.0,0.0,0.0,4,2.0,9,62.5,1.0,1.0,16,5.3,3,0,0.0,546.0,6,12.0,7.7,5,0.0,3,1.0,0,5.0,0,4,0,0,16,4,11,NaN,NaN,Half Back Flank Right,FINAL_TEAM,Half-Back,Back,NaN,NaN,2023,0.000000,-0.260971,0.003827,0.821787,0.531583,0.290204,5.331190,3.690602,1.640589,-0.656076,0.328249,-0.624457,-1.609383,Trent Rivers_2023,0,0.000000,Port Adelaide,Melbourne,Port Adelaide,0.079097,6.111083
26421,202319_Richmond_Hawthorn,Richmond,Trent Cotchin,202319,Trent_Cotchin,MIDFIELDER_FORWARD,CD_I270896,33,183,85,9,RIGHT,VIC,2007.0,2008.0,PEGS (Vic)/Northern U18,2.0,nationalDraft,https://s.

In [50]:
def create_player_ratings(player_values):
    
    total_player_values = player_values.groupby(['Player', 'Team']).sum()[['exp_vaep_value', 'player_value', 'xScore', "Percent_Played"]]
    player_age = player_values.groupby(['Player', 'Team']).mean()['Age']
    total_player_values = pd.concat([total_player_values, player_age], axis=1)
    total_player_values['exp_vaep_rating'] = 100*(total_player_values['exp_vaep_value'] / total_player_values['Percent_Played'])
    total_player_values['xScore_rating'] = 100*(total_player_values['xScore'] / total_player_values['Percent_Played'])
    
    return total_player_values[['Age', "Percent_Played", 'exp_vaep_value', 'xScore', 'exp_vaep_rating', 'xScore_rating']].reset_index()
    # return total_player_values.reset_index()

In [51]:
create_player_ratings(player_values_2023).sort_values(by='exp_vaep_value', ascending=False).head(10)

,Player,Team,Age,Percent_Played,exp_vaep_value,xScore,exp_vaep_rating,xScore_rating
424,Marcus Bontempelli,Western Bulldogs,27.0,1952,257.264062,128.733900,13.179511,6.594974
490,Noah Anderson,Gold Coast,22.0,1941,207.136345,103.656513,10.671630,5.340366
111,Christian Petracca,Melbourne,27.0,1930,204.171447,236.622797,10.578831,12.260249
573,Shai Bolton,Richmond,24.0,1988,197.169424,229.182716,9.917979,11.528306
84,Caleb Serong,Fremantle,22.0,1839,192.770327,60.489718,10.482345,3.289272
105,Charlie Curnow,Carlton,26.0,2152,192.572502,439.139563,8.948536,20.406114
660,Zak Butters,Port Adelaide,22.0,1800,187.002842,78.273519,10.389047,4.348529
376,Lachie Neale,Brisbane Lions,30.0,1989,170.908705,47.100452,8.592695,2.368047
578,Stephen Coniglio,Greater Western Sydney,29.0,1893,170.722975,90.812749,9.018646,4.797293
657,Zach Merrett,Essendon,27.0,1855,168.855597,63.161164,9.102728,3.404915


In [52]:
create_player_ratings(player_values_2023[player_values_2023['Age'] < 25]).sort_values(by='exp_vaep_value', ascending=False).head(10)

,Player,Team,Age,Percent_Played,exp_vaep_value,xScore,exp_vaep_rating,xScore_rating
260,Noah Anderson,Gold Coast,22.000000,1941,207.136345,103.656513,10.671630,5.340366
305,Shai Bolton,Richmond,24.000000,1988,197.169424,229.182716,9.917979,11.528306
39,Caleb Serong,Fremantle,22.000000,1839,192.770327,60.489718,10.482345,3.289272
348,Zak Butters,Port Adelaide,22.000000,1800,187.002842,78.273519,10.389047,4.348529
1,Aaron Naughton,Western Bulldogs,23.000000,1988,156.386599,320.096575,7.866529,16.101437
59,Connor Rozee,Port Adelaide,23.000000,1927,152.692579,141.866704,7.923849,7.362050
47,Chad Warner,Sydney,22.000000,1635,148.855603,99.658565,9.104318,6.095325
226,Matt Rowell,Gold Coast,21.391304,1904,143.793675,36.817296,7.552189,1.933682
157,Jason Horne-Francis,Port Adelaide,19.409091,1682,142.811563,93.474676,8.490580,5.557353
137,Jai Newcombe,Hawthorn,21.181818,1776,141.651502,81.007181,7.975873,4.561215


In [53]:
create_player_ratings(player_values_2023[player_values_2023['Age'] < 21]).sort_values(by='exp_vaep_value', ascending=False).head(10)

,Player,Team,Age,Percent_Played,exp_vaep_value,xScore,exp_vaep_rating,xScore_rating
50,Jason Horne-Francis,Port Adelaide,19.409091,1682,142.811563,93.474676,8.490580,5.557353
85,Nick Daicos,Collingwood,20.000000,1689,132.084004,119.170848,7.820249,7.055704
63,Jye Amiss,Fremantle,19.181818,1719,113.759683,262.411800,6.617783,15.265375
26,Errol Gulden,Sydney,20.000000,1395,106.091932,93.168262,7.605156,6.678728
76,Max Holmes,Geelong,20.000000,1567,96.678760,47.597781,6.169672,3.037510
22,Darcy Wilmot,Brisbane Lions,19.000000,1765,94.967387,38.659646,5.380588,2.190348
81,Mitch Owens,St Kilda,19.000000,1811,94.678027,178.418061,5.227942,9.851908
56,Josh Rachele,Adelaide,20.000000,1605,88.748188,172.367298,5.529482,10.739395
61,Judd McVee,Melbourne,19.173913,1798,87.225161,0.000000,4.851233,0.000000
45,Jake Soligo,Adelaide,20.000000,1610,87.192557,79.705006,5.415687,4.950622
